# Assistant with Function-calling ability

In [9]:
import dotenv, os
import csv
import pandas
from openai import OpenAI

In [10]:
dotenv.load_dotenv(".env")

MODEL="gpt-4o-mini"

In [11]:
client = OpenAI()

## 1. Access order information

We will use a CSV file as a data source to store the order information. The CSV file contains the order number and the order status. We will create a utility to get the order status by order number.

In [12]:
# Let's see how the orders are stored in the csv file
df = pandas.read_csv(os.getcwd()+'/data/orders.csv')
df.head()

,number,status,tracking
0,3001,delivered,1Z999AA10123456784
1,3002,canceled,1Z999AA10123456785
2,3003,delivered,1Z999AA10123456786
3,3004,shipped,1Z999AA10123456787
4,3005,new,NaN


In [13]:
# Get all orders from the csv file
def get_orders():
  with open(os.getcwd()+'/data/orders.csv', mode='r') as infile:
    reader = csv.reader(infile)
    next(reader, None)
    # create orders dict with key first column 
    orders = {str(row[0]):[row[1], row[2]] for row in reader}
    
  return orders

# Get the order information by order number
def get_order_info_by_number(number:str):
  orders = get_orders()
  try:
    return str(orders[str(number)][0])
  except KeyError:
    return "Order not found"

## 2. Create an assistant

Here we add a function name `get_order_info` to the assistant. We create a description, parameters, and required fields for the function. The function will take an order number as a parameter and return the order status.

In [14]:
assistant = client.beta.assistants.create(
  instructions="You are a e-shop order assistant. You can provide information about user's orders",
  model=MODEL,
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_order_info",
        "description": "Get the order status",
        "parameters": {
          "type": "object",
          "properties": {
            "order_number": {
              "type": "string",
              "description": "The order number containing 4 digits and staring from 3"
            }
          },
          "required": ["order_number",]
        }
      }
    },
  ]
)

## 3. Create a thread

In our case the Thread is a conversation between the user and the assistant. 

In [15]:
thread = client.beta.threads.create()

## 4. Send a message

### 4.1 Create a message

In [49]:
# message_text = "I want to know the status of my order 1876238762401"
message_text = "What is the status of order 3001"
# message_text = "i am a cat"

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=message_text
)

### 4.2 Send the message via creating a Run

In [50]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [51]:
# let's check the status of the run
run.status

'requires_action'

In [53]:
# If message is completed, we can show the response to user
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
  
  # That means we can return to step 4.1 and send a new message

## 5. Get the order information

Only if the run.status == "requires_action"

In [54]:
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_order_info":
    
    kargs = eval(tool.function.arguments)
    print(f"passed kwargs are: {kargs}")
    
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": get_order_info_by_number(kargs['order_number'])
    })
      
tool_outputs

passed kwargs are: {'order_number': '3001'}


[{'tool_call_id': 'call_4FWDCW0BAsrbSQzXan5KQKd5', 'output': 'delivered'}]

## 6. Submit the tool outputs

In [55]:
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 

Tool outputs submitted successfully.


In [56]:
# let's check the status of the run
run.status

'completed'

In [60]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
  
   # That means we can return to step 4.1 and send a new message

The status of order 3001 is "delivered." If you need any more information or assistance, just let me know!
